In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction import FeatureHasher
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
columns_to_read = ['YearStart', 'YearEnd', 'LocationDesc', 'Topic', 'Question', 'DataValue']
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SI 670/Final Project/df_filtered.csv', usecols=columns_to_read)

In [ ]:
df.head(2)

,YearStart,YearEnd,LocationDesc,Topic,Question,DataValue
0,2010,2010,Oregon,Cardiovascular Disease,Mortality from heart failure,30.0
1,2011,2011,Alaska,Alcohol,Chronic liver disease mortality,99.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105407 entries, 0 to 105406
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   YearStart     105407 non-null  int64  
 1   YearEnd       105407 non-null  int64  
 2   LocationDesc  105407 non-null  object 
 3   Topic         105407 non-null  object 
 4   Question      105407 non-null  object 
 5   DataValue     71811 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 4.8+ MB


In [ ]:
column_mapping = {
    'LocationDesc': 'state',
    'Topic': 'condition',
    'Question': 'condition_description',
    'DataValue': 'total_count'
}

df = df.rename(columns=column_mapping)

In [ ]:
df['condition'].unique()

array(['Cardiovascular Disease', 'Alcohol', 'Asthma',
       'Chronic Kidney Disease', 'Chronic Obstructive Pulmonary Disease',
       'Diabetes', 'Older Adults', 'Overarching Conditions',
       'Nutrition, Physical Activity, and Weight Status', 'Tobacco'],
      dtype=object)

In [ ]:
states_col_values = list(df['state'].unique())

In [ ]:
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri',
    'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
    'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
    'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

In [ ]:
[element for element in states_col_values if element not in us_states]

['District of Columbia',
 'Puerto Rico',
 'United States',
 'Virgin Islands',
 'Guam']

In [ ]:
len(df[df['state'] == 'United States'])

578

In [ ]:
# drop rows with United States value in the state column
df = df[df['state'] != 'United States']

# drop rows that are not hospitalization or mortality
drop_cats = ['Life expectancy at age 65 years', 'Number of farmers markets per 100,000 residents',
               'Life expectancy at birth', 'Sale of cigarette packs', 'Gini Index of income inequality']
df = df[~df['condition_description'].isin(drop_cats)]

In [ ]:
len(df['state'].unique()) ,len(df['condition'].unique()), len(df['condition_description'].unique())

(54, 8, 29)

In [ ]:
df.shape

(102750, 6)

In [ ]:
df.head()

,YearStart,YearEnd,state,condition,condition_description,total_count
0,2010,2010,Oregon,Cardiovascular Disease,Mortality from heart failure,30.0
1,2011,2011,Alaska,Alcohol,Chronic liver disease mortality,99.0
2,2016,2016,Alaska,Alcohol,Chronic liver disease mortality,58.0
3,2019,2019,Alaska,Alcohol,Chronic liver disease mortality,110.0
4,2016,2016,Alabama,Alcohol,Chronic liver disease mortality,118.0


In [ ]:
unique_categories_per_column = {'state': 55, 'condition': 30, 'condition_description': 20}

feature_hashers = {column: FeatureHasher(n_features=min(10, 2 * num_categories), input_type='string')
                   for column, num_categories in unique_categories_per_column.items()}

hashed_feature_dfs = []
for column, hasher in feature_hashers.items():
    hashed_features = hasher.transform([[str(val)] for val in df[column]])
    hashed_feature_df = pd.DataFrame(hashed_features.toarray(), columns=[f'{column}_hash_{i}' for i in range(hashed_features.shape[1])])
    hashed_feature_dfs.append(hashed_feature_df)

df_hashed = pd.concat([df] + hashed_feature_dfs, axis=1)

In [ ]:
df_hashed.shape

(102750, 91)

In [ ]:
df_hashed.columns[df_hashed.isnull().any()]

Index(['total_count'], dtype='object')

In [ ]:
imputer = IterativeImputer(random_state=42, max_iter=10)
imputed_df = pd.DataFrame(imputer.fit_transform(df_hashed), columns=df_hashed.columns, index=df_hashed.index)
print("Shape after imputation:", imputed_df.shape)

Shape after imputation: (102750, 91)


In [ ]:
imputed_df.columns[imputed_df.isnull().any()]

(102750, 91)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_values = scaler.fit_transform(imputed_df['total_count'].values.reshape(-1,1))
imputed_df['total_count'] = scaled_values.flatten()

In [ ]:
imputed_df.to_csv('cleaned_data.csv', index=False)